### Numpy code

In [1]:
import numpy as np

# Example: Large matrices (adjust size as needed)
n = 7000  # For very large matrices, ensure you have enough RAM
A = np.random.rand(n, n).astype(np.float32)
B = np.random.rand(n, n).astype(np.float32)

C = np.dot(A, B)  # warm-up and Matrix multiplication

%timeit -r 2 -o np.dot(A, B)

print(f"Result shape: {C.shape}")
print(f"Result type: {C.dtype}")


1.06 s ± 6.16 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result shape: (7000, 7000)
Result type: float32


## Multiplicación con PyTorch

In [2]:
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

A_torch = torch.from_numpy(A).to(device)
B_torch = torch.from_numpy(B).to(device)

# warm-up
C_torch = torch.matmul(A_torch, B_torch)
if torch.cuda.is_available():
    torch.cuda.synchronize()

start = time.time()
C_torch = torch.matmul(A_torch, B_torch)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()

print("Tiempo PyTorch:", end - start)

C_result = C_torch.cpu().numpy()
print(C_result.shape)
print(C_result.dtype)

diff = np.max(np.abs(C - C_result))
print("Diferencia máxima:", diff)

Using device: cuda


/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 1080 which is of cuda capability 6.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 1080 with CUDA capability sm_61 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 1080 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  wa

164 ms ± 388 μs per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Result shape: (7000, 7000)
Result type: float32
Max difference with NumPy: 1.21e-02


## Cálculo de π con PyTorch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def calc_pi_pytorch(N, device):
    x = torch.rand(N, device=device) * 2 - 1
    y = torch.rand(N, device=device) * 2 - 1
    inside = (x * x + y * y) < 1.0
    M = inside.float().sum().item()
    return 4 * M / N

sizes = [1000000, 10000000]

for N in sizes:
    if torch.cuda.is_available():
        torch.cuda.synchronize()

    start = time.time()
    pi_value = calc_pi_pytorch(N, device)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    end = time.time()

    print("N =", N)
    print("pi =", pi_value)
    print("error =", abs(pi_value - np.pi))
    print("tiempo =", end - start)

AcceleratorError: CUDA error: no kernel image is available for execution on the device
Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## NumPy (CPU)

In [ ]:
def calc_pi_numpy(N):
    x = np.random.uniform(-1, 1, N)
    y = np.random.uniform(-1, 1, N)
    inside = (x * x + y * y) < 1.0
    return 4 * np.sum(inside) / N

N = 10000000

start = time.time()
pi_numpy = calc_pi_numpy(N)
end = time.time()

print("pi NumPy =", pi_numpy)
print("tiempo NumPy =", end - start)


## Comparación CPU vs GPU

In [ ]:
# NumPy
start = time.time()
C_numpy = np.dot(A, B)
end = time.time()
time_cpu = end - start
print("Tiempo NumPy:", time_cpu)

# PyTorch
if torch.cuda.is_available():
    torch.cuda.synchronize()
    start = time.time()
    C_torch = torch.matmul(A_torch, B_torch)
    torch.cuda.synchronize()
    end = time.time()
    time_gpu = end - start
    print("Tiempo PyTorch GPU:", time_gpu)
    print("Speedup:", time_cpu / time_gpu)

## Análisis de resultados

En la multiplicación de matrices de tamaño 7000 × 7000 se observa una diferencia clara entre la ejecución en CPU con NumPy y la ejecución en GPU con PyTorch. NumPy tarda aproximadamente 0.675 segundos por ejecución, mientras que PyTorch en GPU reduce el tiempo a unos 0.060 segundos. Esto supone una aceleración cercana a 11 veces, lo que demuestra que la GPU es mucho más eficiente para operaciones matriciales grandes debido a su alto paralelismo. La diferencia máxima entre los resultados de NumPy y PyTorch es pequeña y se explica por el uso de precisión float32 y por el distinto orden de las operaciones en GPU, por lo que el resultado puede considerarse correcto.

En el cálculo de π mediante el método de Monte Carlo con PyTorch en GPU, para un millón de puntos el valor obtenido presenta un error relativamente mayor y un tiempo de ejecución más alto, debido principalmente a la sobrecarga inicial y a la generación de números aleatorios. Sin embargo, al aumentar el número de puntos a diez millones, el error disminuye notablemente y el tiempo de ejecución se reduce de forma drástica, mostrando que la GPU aprovecha mejor su paralelismo cuando el volumen de trabajo es grande.

La comparación directa entre PyTorch en GPU y NumPy en CPU para el cálculo de π con diez millones de puntos confirma esta tendencia. NumPy tarda alrededor de 0.179 segundos, mientras que PyTorch en GPU completa el cálculo en apenas 0.0016 segundos, obteniendo una aceleración significativa. 

675 ms ± 1.78 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Result shape: (7000, 7000)

Result type: float32
________________________________________________________________________
cuda

Tiempo PyTorch: 0.060648202896118164

(7000, 7000)

float32

Diferencia máxima: 0.011962891
________________________________________________________________________
cuda

N = 1000000

pi = 3.144312

error = 0.002719346410207102

tiempo = 1.1935160160064697
________________________________________________________________________
N = 10000000

pi = 3.1425156

error = 0.0009229464102067375

tiempo = 0.0016279220581054688

pi NumPy = 3.1423724

tiempo NumPy = 0.1786792278289795
________________________________________________________________________
Tiempo NumPy: 0.6612417697906494

Tiempo PyTorch GPU: 0.060597896575927734

Speedup: 10.91195911333538